In [2]:
from collections import defaultdict
import pickle
from datetime import datetime, timedelta

import pandas as pd

### Chicago
Data source for Chicago:

https://www.chicago.gov/city/en/sites/covid-19/home/latest-data.html

Create a dict of the form {days from Feb 29: number of deaths}, and save it as a pandas DataFrame starting with Feb 1 to the end of the dict.

In [73]:
chi_death_dict = defaultdict(int, {16: 1, 17: 1, 18: 1, 19: 2, 21: 1,
                                   22: 3, 23: 3, 24: 2, 25: 5, 26: 2,
                                   27: 7, 28: 4, 29: 21, 30: 12,
                                   31: 13, 32: 23, 33: 11, 34: 24,
                                   35: 18, 36: 30, 37: 18, 38: 22,
                                   39: 33, 40: 20, 41: 26, 42: 27,
                                   43: 29, 44: 24, 45: 20, 46: 17})

In [74]:
chi_last_day = max(chi_death_dict.keys())

In [75]:
chi_index = [datetime(2020, 2, 1) + timedelta(days=day) for day in range(29 + chi_last_day)]
chi_deaths =[chi_death_dict[day - 28] for day in range(29 + chi_last_day)]

In [76]:
chi_death_df = pd.DataFrame(chi_deaths, index=chi_index, columns=['deaths'])

In [77]:
# Inspect that last few days
chi_death_df.tail()

,deaths
2020-04-11,27
2020-04-12,29
2020-04-13,24
2020-04-14,20
2020-04-15,17


In [78]:
chi_death_df.to_pickle('../data/chi_deaths.pickle')

### Illinois
Data source for Illinois:

http://www.dph.illinois.gov/covid19/covid19-statistics

`il_cum_death_list` is the list of cumulative deaths on each day, starting on March 16. Again, save it as a pandas DataFrame starting with Feb 1 to the end of the dict.

In [79]:
il_cum_death_list = [0, 1, 3, 4, 5, 6, 9, 12, 16, 19, 26, 34, 47,
                     65, 72, 99, 141, 157, 210, 243, 274, 307, 380,
                     462, 528, 596, 677, 720, 794, 868, 948, 1072]

In [80]:
# New deaths from March 17
il_death_list = [now - prev 
                 for now, prev in zip(il_cum_death_list[1:],
                                      il_cum_death_list[:-1])]

In [81]:
il_death_dict = defaultdict(int, {index + 17: diff
                                  for index, diff in enumerate(il_death_list)})

In [82]:
il_num_days = len(il_death_list)

In [83]:
num_days = len(il_death_list)

In [84]:
il_index = [datetime(2020, 2, 1) + timedelta(days=day) for day in range(29 + 16 + num_days)]
il_deaths =[il_death_dict[day - 28] for day in range(29 + 16 + num_days)]

In [85]:
il_death_df = pd.DataFrame(il_deaths, index=il_index, columns=['deaths'])

In [86]:
# Inspect that last few days
il_death_df.tail()

,deaths
2020-04-12,43
2020-04-13,74
2020-04-14,74
2020-04-15,80
2020-04-16,124


In [87]:
il_death_df.to_pickle('../data/il_deaths.pickle')